In [59]:
import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from hashlib import md5
from typing import List
from google.cloud import bigquery
from google.oauth2 import service_account

key_path = "/root/.creds/kphil-sa.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)


# **** SETUP ****

DATA_DIR = "../data/"
DEFAULT_BOSTON_CRIME_FILE = os.path.join(DATA_DIR, "boston_crime_data.csv")
DEFAULT_PORTLAND_CRIME_FILE = os.path.join(DATA_DIR, "portland_crime_data.csv")
PROJECT_NAME = "team-week2"
BOSTON_DATASET_NAME = "boston"
PORTLAND_DATASET_NAME = "portland"


# **** TABLE SCHEMAS ****

TABLE_METADATA = {
    'boston_crime': {
        'table_name': 'boston_crime',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('incident_number', 'string', mode='REQUIRED'),
            bigquery.SchemaField('offense_code', 'int64', mode='REQUIRED'),
            bigquery.SchemaField('offense_code_group', 'string', mode='REQUIRED'),
            bigquery.SchemaField('offense_description', 'string', mode='NULLABLE'),
            bigquery.SchemaField('district', 'string', mode='REQUIRED'),
            bigquery.SchemaField('reporting_area', 'int64',mode='NULLABLE'),
            bigquery.SchemaField('shooting', 'string',mode='NULLABLE'),
            bigquery.SchemaField('occurred_on_date','datetime64', mode='REQUIRED'),
            bigquery.SchemaField('year', 'int64', mode='REQUIRED'),
            bigquery.SchemaField('month', 'int64', mode='REQUIRED'),
            bigquery.SchemaField('day_of_week', 'string', mode='REQUIRED'),
            bigquery.SchemaField('hour', 'int64', mode='REQUIRED'),
            bigquery.SchemaField('ucr_part','string', mode='NULLABLE'),
            bigquery.SchemaField('street', 'string', mode='REQUIRED'),
            bigquery.SchemaField('lat','float64', mode='REQUIRED'),
            bigquery.SchemaField('long', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('location', 'object', mode='REQUIRED')
        ]
    },
    'portland_crime': {
        'table_name': 'portland_crime',
        'schema': [
            bigquery.SchemaField('address', 'string', mode='REQUIRED'),
            bigquery.SchemaField('case_number', 'string', mode='REQUIRED'),
            bigquery.SchemaField('crime_against', 'string', mode='REQUIRED'),
            bigquery.SchemaField('neighborhood', 'string', mode='REQUIRED'),
            bigquery.SchemaField('number_of_records', 'int64', mode='REQUIRED'),
            bigquery.SchemaField('occur_date', 'datetime64',mode='REQUIRED'),
            bigquery.SchemaField('occur_month_year', 'datetime64',mode='REQUIRED'),
            bigquery.SchemaField('occur_time','datetime64', mode='REQUIRED'),
            bigquery.SchemaField('offense_category', 'string', mode='REQUIRED'),
            bigquery.SchemaField('offense_count', 'int64', mode='REQUIRED'),
            bigquery.SchemaField('offense_type', 'string', mode='REQUIRED'),
            bigquery.SchemaField('opendatalat', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('opendatalon', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('opendatax','float64', mode='NULLABLE'),
            bigquery.SchemaField('opendatay', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('report_date','datetime64', mode='REQUIRED'),
            bigquery.SchemaField('reportmonthyear', 'object', mode='REQUIRED')
        ]
    }
}


# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level


# **** BIGQUERY CLIENT ****
logger.debug(f"Creating bigquery client")

logger.info(f"Setup Completed")

[DEBUG][2023-01-23 05:46:32,898][1634121891:0093] : Creating bigquery client
[INFO ][2023-01-23 05:46:32,904][1634121891:0095] : Setup Completed


In [7]:
# Boston crime data file name
filename = DEFAULT_BOSTON_CRIME_FILE
logger.debug(f"attempting to process: {filename}")

# Check if file exists
assert os.path.exists(filename), f"Data file does not exists: '{filename}'"
# check if the file contains any data. Header row alone is 78 bytes. size > 78
assert os.path.getsize(filename) > 78, f"Data file size incorrect; does not seem to contain data: '{filename}'"

# Load boston crime file into dataframe
df = pd.read_csv(filename, header=0, encoding='windows-1252')
logger.info(f"loaded {len(df.index)} rows from: {filename}")

df.columns = df.columns.str.lower()

# *** perform check at the end ***
# check schema: contains all expected columns?
expected_columns = ['incident_number', 'offense_code', 'offense_code_group', 'offense_description', 'district', 'reporting_area', 'shooting', 'occurred_on_date', 'year', 'month', 'day_of_week', 'hour', 'ucr_part', 'street', 'lat', 'long', 'location']
for col in expected_columns:
    assert col in list(df.columns), f"Data file missing required column: {col}"

# assign & remember boston crime dataframe
boston_crime_df = df
boston_crime_df.head()

[DEBUG][2023-01-23 03:42:58,952][860230781:0003] : attempting to process: ../data/boston_crime_data.csv
[INFO ][2023-01-23 03:43:00,230][860230781:0012] : loaded 319073 rows from: ../data/boston_crime_data.csv


,incident_number,offense_code,offense_code_group,offense_description,district,reporting_area,shooting,occurred_on_date,year,month,day_of_week,hour,ucr_part,street,lat,long,location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [8]:
# Create dataset
dataset_id = f"{PROJECT_NAME}.{BOSTON_DATASET_NAME}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)

logger.info(f"Created Boston dataset: {dataset.full_dataset_id}")

[INFO ][2023-01-23 03:44:27,923][3283432699:0007] : Created Boston dataset: team-week2:boston


In [9]:
df.dtypes

incident_number         object
offense_code             int64
offense_code_group      object
offense_description     object
district                object
reporting_area          object
shooting                object
occurred_on_date        object
year                     int64
month                    int64
day_of_week             object
hour                     int64
ucr_part                object
street                  object
lat                    float64
long                   float64
location                object
dtype: object

In [31]:
# Converting datatypes of columns
df[['incident_number', 'offense_code_group', 'offense_description', 'district', 'shooting', 'ucr_part', 'street', 'day_of_week']] = df[['incident_number', 'offense_code_group', 'offense_description', 'district', 'shooting', 'ucr_part', 'street', 'day_of_week']].astype('string')

df['occurred_on_date'] = df['occurred_on_date'].astype('datetime64[ns]')

df.dtypes

incident_number                string
offense_code                    int64
offense_code_group             string
offense_description            string
district                       string
reporting_area                 object
shooting                       string
occurred_on_date       datetime64[ns]
year                            int64
month                           int64
day_of_week                    string
hour                            int64
ucr_part                       string
street                         string
lat                           float64
long                          float64
location                       object
dtype: object

In [39]:
# Load Boston crime dataframe to BigQuery
table_name = f"{PROJECT_NAME}.{BOSTON_DATASET_NAME}.{TABLE_METADATA['boston_crime']['table_name']}"

job_config = bigquery.LoadJobConfig(
    create_disposition = 'CREATE_IF_NEEDED', 
    write_disposition = 'WRITE_TRUNCATE',
    autodetect = True
    )
logger.info(f"loading table: '{table_name}'")
job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
job.result()        # wait for the job to finish
# get the resulting table
table = client.get_table(table_name)
logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")

[INFO ][2023-01-23 04:42:56,292][1981220102:0009] : loading table: 'team-week2.boston.boston_crime'
[INFO ][2023-01-23 04:43:27,128][1981220102:0014] : loaded 319073 rows into team-week2:boston.boston_crime


In [42]:
# Portland crime data file name
filename = DEFAULT_PORTLAND_CRIME_FILE
logger.debug(f"attempting to process: {filename}")

# Check if file exists
assert os.path.exists(filename), f"Data file does not exists: '{filename}'"
# check if the file contains any data. Header row alone is 78 bytes. size > 78
assert os.path.getsize(filename) > 78, f"Data file size incorrect; does not seem to contain data: '{filename}'"

# Load portland crime file into dataframe
df = pd.read_csv(filename, header=0)
logger.info(f"loaded {len(df.index)} rows from: {filename}")

df.columns = df.columns.str.lower().str.replace(' ', '_')

# *** perform check at the end ***
# check schema: contains all expected columns?
expected_columns = ['address', 'case_number', 'crime_against', 'neighborhood', 'number_of_records', 'occur_date', 'occur_month_year', 'occur_time', 'offense_category', 'offense_count', 'offense_type', 'opendatalat', 'opendatalon', 'opendatax', 'opendatay', 'report_date', 'reportmonthyear']
for col in expected_columns:
    assert col in list(df.columns), f"Data file missing required column: {col}"

# assign & remember boston crime dataframe
portland_crime_df = df
portland_crime_df.head()

[DEBUG][2023-01-23 05:23:55,963][3334852744:0003] : attempting to process: ../data/portland_crime_data.csv
[INFO ][2023-01-23 05:23:56,854][3334852744:0012] : loaded 202552 rows from: ../data/portland_crime_data.csv


,address,case_number,crime_against,neighborhood,number_of_records,occur_date,occur_month_year,occur_time,offense_category,offense_count,offense_type,opendatalat,opendatalon,opendatax,opendatay,report_date,reportmonthyear
0,3600 BLOCK OF SE KNAPP ST,17-902332,Property,Eastmoreland,1,2/20/2017,2/1/2017,0,Larceny Offenses,1,Theft From Motor Vehicle,45.470545,-122.625298,7656952.0,664817.0,2/20/2017,February 2017
1,3600 BLOCK OF SE LAMBERT ST,17-902346,Property,Eastmoreland,1,2/20/2017,2/1/2017,30,Larceny Offenses,1,Theft From Motor Vehicle,45.467028,-122.625272,7656925.0,663535.0,2/20/2017,February 2017
2,7200 BLOCK OF SE 32ND AVE,17-902450,Property,Eastmoreland,1,2/21/2017,2/1/2017,2345,Larceny Offenses,1,Theft From Motor Vehicle,45.471859,-122.630327,7655675.0,665330.0,2/22/2017,February 2017
3,6500 BLOCK OF SE 32ND AVE,17-902495,Property,Eastmoreland,1,2/21/2017,2/1/2017,2350,Larceny Offenses,1,Theft From Motor Vehicle,45.475196,-122.630444,7655677.0,666547.0,2/22/2017,February 2017
4,500 BLOCK OF N DIXON ST,17-901848,Property,Eliot,1,12/21/2016,12/1/2016,1330,Larceny Offenses,1,Theft From Motor Vehicle,45.534551,-122.671730,7645672.0,688462.0,2/11/2017,February 2017


In [43]:
# Create dataset
dataset_id = f"{PROJECT_NAME}.{PORTLAND_DATASET_NAME}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)

logger.info(f"Created Portland dataset: {dataset.full_dataset_id}")

[INFO ][2023-01-23 05:25:28,017][1718675616:0007] : Created Portland dataset: team-week2:portland


In [44]:
df.dtypes

address               object
case_number           object
crime_against         object
neighborhood          object
number_of_records      int64
occur_date            object
occur_month_year      object
occur_time             int64
offense_category      object
offense_count          int64
offense_type          object
opendatalat          float64
opendatalon          float64
opendatax            float64
opendatay            float64
report_date           object
reportmonthyear       object
dtype: object

In [58]:
df[['address','case_number', 'crime_against', 'neighborhood', 'offense_category', 'offense_type']] = df[['address', 'case_number', 'crime_against', 'neighborhood', 'offense_category', 'offense_type']].astype('string')

df[['occur_date', 'report_date', 'reportmonthyear', 'occur_month_year']] = df[['occur_date','report_date', 'reportmonthyear', 'occur_month_year']].astype('datetime64[ns]')

df.dtypes

address                      string
case_number                  string
crime_against                string
neighborhood                 string
number_of_records             int64
occur_date           datetime64[ns]
occur_month_year     datetime64[ns]
occur_time                    int64
offense_category             string
offense_count                 int64
offense_type                 string
opendatalat                 float64
opendatalon                 float64
opendatax                   float64
opendatay                   float64
report_date          datetime64[ns]
reportmonthyear      datetime64[ns]
dtype: object